In [1]:
%tensorflow_version 1.x 

TensorFlow 1.x selected.


In [0]:
import pandas as pd
import numpy as np

In [23]:
df = pd.DataFrame()
df = pd.read_csv('/content/drive/My Drive/movie_data.csv', encoding='utf-8')
df.head()

,review,sentiment
0,I went and saw this movie last night after bei...,1
1,Actor turned director Bill Paxton follows up h...,1
2,As a recreational golfer with some knowledge o...,1
3,"I saw this film in a sneak preview, and it is ...",1
4,Bill Paxton has taken the true story of the 19...,1


In [0]:
xtrain = df.loc[:24999,'review'].values
ytrain = df.loc[:24999,'sentiment'].values
xtest = df.loc[25000:,'review'].values
ytest = df.loc[25000:,'sentiment'].values

## SENTIMENT CLASSIFICAITON

In [0]:
from tensorflow.python.keras.preprocessing.text import Tokenizer
from tensorflow.python.keras.preprocessing.sequence import pad_sequences

In [0]:
x = xtrain + xtest
y = ytrain + ytest

In [0]:
tokenizer_obj = Tokenizer()
tokenizer_obj.fit_on_texts(x)

In [0]:
#pad sequences
maxlength = 100

In [0]:
#define vocabulary size
vocabsize = len(tokenizer_obj.word_index) + 1

In [0]:
xtrain_tokens = tokenizer_obj.texts_to_sequences(xtrain)
xtest_tokens = tokenizer_obj.texts_to_sequences(xtest)

In [0]:
xtrain_pad = pad_sequences(xtrain_tokens, maxlen=maxlength, padding='post')
xtest_pad = pad_sequences(xtest_tokens, maxlen=maxlength, padding='post')

In [49]:
from keras.models import Sequential
from keras.layers import Dense, Embedding, LSTM, GRU
from keras.layers.embeddings import Embedding

EMBEDDDING_DIM = 100

print('Build model....')
model = Sequential()
model.add(Embedding(vocabsize, EMBEDDDING_DIM, input_length = maxlength))
model.add(GRU(units=32, dropout=0.2, recurrent_dropout=0.2))
model.add(Dense(1, activation='sigmoid'))

Build model....


In [39]:
Embedding(vocabsize, EMBEDDDING_DIM, input_length=maxlength)

In [0]:
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

In [51]:
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, 100, 100)          12560200  
_________________________________________________________________
gru_1 (GRU)                  (None, 32)                12768     
_________________________________________________________________
dense_1 (Dense)              (None, 1)                 33        
Total params: 12,573,001
Trainable params: 12,573,001
Non-trainable params: 0
_________________________________________________________________


In [52]:
print('Train...')

model.fit(xtrain_pad, ytrain, batch_size=128, epochs=5, validation_data=(xtest_pad, ytest), verbose=2)

Train...
Train on 25000 samples, validate on 25000 samples
Epoch 1/5
 - 51s - loss: 0.6077 - accuracy: 0.6964 - val_loss: 0.5756 - val_accuracy: 0.6916
Epoch 2/5
 - 51s - loss: 0.3973 - accuracy: 0.8268 - val_loss: 0.5492 - val_accuracy: 0.7150
Epoch 3/5
 - 50s - loss: 0.3248 - accuracy: 0.8675 - val_loss: 0.5670 - val_accuracy: 0.7044
Epoch 4/5
 - 51s - loss: 0.2922 - accuracy: 0.8818 - val_loss: 0.5822 - val_accuracy: 0.7012
Epoch 5/5
 - 53s - loss: 0.2228 - accuracy: 0.9158 - val_loss: 0.5950 - val_accuracy: 0.7048


In [53]:
#Let us test some  samples
sample_1 = "This movie is fantastic! I really like it because it is so good!"
sample_2 = "Director has visualised the story line in detail"
sample_3 = "Awesome movie."
sample_4 = "Not to my taste, will skip and watch another movie"
sample_5 = "if you like action, then this movie might be good for you."
sample_6 = "Bad movie!"
sample_7 = "Not a good movie!"
sample_8 = "This movie really sucks! Can I get my money back please?"
samples = [sample_1, sample_2, sample_3, sample_4, sample_5, sample_6, sample_7, sample_8]

samples_tokens = tokenizer_obj.texts_to_sequences(samples)
samples_tokens_pad = pad_sequences(samples_tokens, maxlen=maxlength)

#predict
model.predict(x=samples_tokens_pad)

array([[0.89168656],
       [0.8835851 ],
       [0.9091923 ],
       [0.04754969],
       [0.60370624],
       [0.11528671],
       [0.53573734],
       [0.05465543]], dtype=float32)

## Train word2vec Embedding

In [0]:
%matplotlib inline

In [55]:
import matplotlib.pyplot as plt
import nltk
nltk.download('punkt')
nltk.download('stopwords')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [0]:
import string
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords

In [0]:
review_lines=list()
lines = df['review'].values.tolist()

for line in lines:
  tokens = word_tokenize(line)
  #converting into lower case
  tokens = [w.lower() for w in tokens]
  #removing punctuation
  table = str.maketrans('','',string.punctuation)
  stripped = [w.translate(table) for w in tokens]
  #remove remaining tokens that are not alphabetic
  words = [word for word in stripped if word.isalpha()]
  #filter out stop words
  stop_words = set(stopwords.words('english'))
  words = [w for w in words if not w in stop_words]
  review_lines.append(words)

In [58]:
len(review_lines)

50000

In [63]:
import gensim

#train word2vec model
model = gensim.models.Word2Vec(sentences=review_lines, size=EMBEDDDING_DIM, window=5, workers=4, min_count=1)

#vocab size
words = list(model.wv.vocab)
print('Vocabulary size: %d' % len(words))

Vocabulary size: 134156


In [64]:
model.wv.most_similar('horrible')

/usr/local/lib/python3.6/dist-packages/gensim/matutils.py:737: FutureWarning: Conversion of the second argument of issubdtype from `int` to `np.signedinteger` is deprecated. In future, it will be treated as `np.int64 == np.dtype(int).type`.
  if np.issubdtype(vec.dtype, np.int):


[('terrible', 0.9191679954528809),
 ('awful', 0.8707277774810791),
 ('atrocious', 0.7795190811157227),
 ('dreadful', 0.771564245223999),
 ('horrendous', 0.7708115577697754),
 ('pathetic', 0.7673578858375549),
 ('lousy', 0.7419604063034058),
 ('sucks', 0.7410047054290771),
 ('horrid', 0.7321884036064148),
 ('ridiculous', 0.7216708660125732)]

In [65]:
model.wv.most_similar('good')

/usr/local/lib/python3.6/dist-packages/gensim/matutils.py:737: FutureWarning: Conversion of the second argument of issubdtype from `int` to `np.signedinteger` is deprecated. In future, it will be treated as `np.int64 == np.dtype(int).type`.
  if np.issubdtype(vec.dtype, np.int):


[('decent', 0.787473738193512),
 ('great', 0.7331076264381409),
 ('ok', 0.6903510093688965),
 ('fine', 0.6828722357749939),
 ('alright', 0.6828575134277344),
 ('okay', 0.6733218431472778),
 ('excellent', 0.67041015625),
 ('bad', 0.6686191558837891),
 ('nice', 0.6590350866317749),
 ('solid', 0.6456297039985657)]

In [66]:
model.wv.most_similar('beauty')

/usr/local/lib/python3.6/dist-packages/gensim/matutils.py:737: FutureWarning: Conversion of the second argument of issubdtype from `int` to `np.signedinteger` is deprecated. In future, it will be treated as `np.int64 == np.dtype(int).type`.
  if np.issubdtype(vec.dtype, np.int):


[('beautiful', 0.6723564863204956),
 ('portrait', 0.6509132385253906),
 ('simplicity', 0.6463258266448975),
 ('mediterraneans', 0.644288957118988),
 ('sensual', 0.6384891867637634),
 ('fleshly', 0.6334782242774963),
 ('poetic', 0.6330362558364868),
 ('messagethe', 0.6329184770584106),
 ('memorability', 0.632462739944458),
 ('passion', 0.6273937225341797)]

In [67]:
#result of semantically reasonable word vectors
model.wv.most_similar_cosmul(positive=['queen','king'], negative=['man'])

[('moppets', 1.4002612829208374),
 ('romeo', 1.3400325775146484),
 ('littlebuddy', 1.3373264074325562),
 ('linklaters', 1.3069872856140137),
 ('brinkman', 1.2915101051330566),
 ('juliet', 1.2893790006637573),
 ('melieslike', 1.2708055973052979),
 ('les', 1.2425140142440796),
 ('la', 1.24093496799469),
 ('costello', 1.2406177520751953)]

In [68]:
#odd word out
print(model.wv.doesnt_match('woman king queen movie'.split()))

movie


/usr/local/lib/python3.6/dist-packages/gensim/models/keyedvectors.py:895: FutureWarning: arrays to stack must be passed as a "sequence" type such as list or tuple. Support for non-sequence iterables such as generators is deprecated as of NumPy 1.16 and will raise an error in the future.
  vectors = vstack(self.word_vec(word, use_norm=True) for word in used_words).astype(REAL)
/usr/local/lib/python3.6/dist-packages/gensim/matutils.py:737: FutureWarning: Conversion of the second argument of issubdtype from `int` to `np.signedinteger` is deprecated. In future, it will be treated as `np.int64 == np.dtype(int).type`.
  if np.issubdtype(vec.dtype, np.int):


In [69]:
#save model
filename = 'imdb_embedding_word2vec.txt'
model.wv.save_word2vec_format(filename, binary=False)

/usr/local/lib/python3.6/dist-packages/smart_open/smart_open_lib.py:253: UserWarning: This function is deprecated, use smart_open.open instead. See the migration notes for details: https://github.com/RaRe-Technologies/smart_open/blob/master/README.rst#migrating-to-the-new-open-function
  'See the migration notes for details: %s' % _MIGRATION_NOTES_URL


In [0]:
import os
embeddings_index = {}
f = open(os.path.join('','imdb_embedding_word2vec.txt'), encoding='utf-8')
for line in f:
  values = line.split()
  word = values[0]
  coefs = np.asarray(values[1:])
  embeddings_index[word] = coefs
f.close()

In [0]:
#vectorize the text samples into a 2D integer tensor
tokenizer_obj.fit_on_texts(review_lines)
sequences = tokenizer_obj.texts_to_sequences(review_lines)

In [72]:
#pad sequeces
word_index = tokenizer_obj.word_index
print('Found %s unique tokens.'% len(word_index))

Found 163766 unique tokens.


In [74]:
review_pad = pad_sequences(sequences, maxlen=maxlength)
sentiment = df['sentiment'].values
print('Shape of review tensor:', review_pad.shape)
print('Shape of sentiment tensor:', sentiment.shape)

Shape of review tensor: (50000, 100)
Shape of sentiment tensor: (50000,)


In [0]:
num_words = len(word_index) + 1
embedding_matrix = np.zeros((num_words, EMBEDDDING_DIM))

for word, i in word_index.items():
  if i > num_words:
    continue
  embedding_vector = embeddings_index.get(word)
  if embedding_vector is not None:
    embedding_matrix[i] = embedding_vector

In [76]:
print(num_words)

163767


In [0]:
from keras.initializers import Constant

In [0]:
model = Sequential()
embedding_layer = Embedding(num_words, EMBEDDDING_DIM, embeddings_initializer=Constant(embedding_matrix), 
                            input_length=maxlength, trainable=False)
model.add(embedding_layer)
model.add(GRU(units=32, dropout=0.2, recurrent_dropout=0.2))
model.add(Dense(1, activation='sigmoid'))


In [0]:
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

In [80]:
model.summary()

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_2 (Embedding)      (None, 100, 100)          16376700  
_________________________________________________________________
gru_2 (GRU)                  (None, 32)                12768     
_________________________________________________________________
dense_2 (Dense)              (None, 1)                 33        
Total params: 16,389,501
Trainable params: 12,801
Non-trainable params: 16,376,700
_________________________________________________________________


In [0]:
VALIDATION_SPLIT = 0.2

In [0]:
indices = np.arange(review_pad.shape[0])
np.random.shuffle(indices)
review_pad = review_pad[indices]
sentiment = sentiment[indices]
num_validation_samples = int(VALIDATION_SPLIT * review_pad.shape[0])

In [0]:
x_train_pad = review_pad[:-num_validation_samples]
y_train = sentiment[:-num_validation_samples]
x_test_pad = review_pad[-num_validation_samples:]
y_test = sentiment[-num_validation_samples:]

In [84]:
print('Shape of x_train_pad tensor:', x_train_pad.shape)
print('Shape of y_train tensor:', y_train.shape)


Shape of x_train_pad tensor: (40000, 100)
Shape of y_train tensor: (40000,)


In [85]:
print('Shape of x_test_pad tensor:', x_test_pad.shape)
print('Shape of y_test tensor:', y_test.shape)

Shape of x_test_pad tensor: (10000, 100)
Shape of y_test tensor: (10000,)


In [88]:
print('Train..')
model.fit(x_train_pad, y_train, batch_size=128, epochs=5, validation_data=(x_test_pad, y_test), verbose=2)

Train..
Train on 40000 samples, validate on 10000 samples
Epoch 1/5
 - 27s - loss: 1002.7482 - accuracy: 0.5313 - val_loss: 0.7024 - val_accuracy: 0.5418
Epoch 2/5
 - 27s - loss: 16.1447 - accuracy: 0.5329 - val_loss: 0.6990 - val_accuracy: 0.5493
Epoch 3/5
 - 27s - loss: 0.7216 - accuracy: 0.5337 - val_loss: 0.6977 - val_accuracy: 0.5501
Epoch 4/5
 - 27s - loss: 1.1066 - accuracy: 0.5335 - val_loss: 0.6970 - val_accuracy: 0.5509
Epoch 5/5
 - 27s - loss: 23.6599 - accuracy: 0.5386 - val_loss: 0.6956 - val_accuracy: 0.5575
